# Análisis y visualización de datos con Python
# 5. Remodelación de datos


---


Una vez que los datos han sido limpiados y estructurados, la siguiente etapa en el proceso de análisis es la **manipulación y transformación de datos**. En esta fase, reorganizamos la información para que sea adecuada para análisis más avanzados o para la creación de visualizaciones. La manipulación de datos va más allá de la simple limpieza; se centra en la agregación, la combinación de conjuntos de datos y la reestructuración de la información.

En este notebook, continuaremos trabajando con el conjunto de datos de muertes asociadas a causas respiratorias, que ya hemos seleccionado y limpiado a partir del EDR del INEGI en los ejercicios anteriores. Nos enfocaremos en transformar este conjunto de datos para explorar patrones de mortalidad por causas respiratorias en 2023 desde diferentes perspectivas.

Los temas que exploraremos son:
* **Combinación de DataFrames**: Uniremos información de múltiples tablas usando `merge()` y `concat()`.
* **Tablas pivote**: Crearemos tablas resumen para analizar datos con múltiples variables categóricas.
* **Agregación de datos**: Utilizaremos `groupby()` para calcular estadísticas resumidas (como el conteo de muertes, promedios de edad, etc.) para diferentes grupos de interés.

Para empezar, cargaremos el conjunto de datos limpio que guardamos en formato Pickle, ya que este formato conserva los tipos de datos y la estructura del DataFrame, lo que nos ahorrará tiempo.

In [ ]:
import pandas as pd
import numpy as np

df_respiratorio = pd.read_pickle('data_clean/defunciones_respiratorias_2023.pkl')
df_respiratorio

In [1]:
import pandas as pd

filename = 'data_raw/CNB_DOB_BPGS_Respuesta_Solicitud 332163723000249.xlsx'
df = pd.read_excel(filename, #nombre del archivo
                   sheet_name="HBO", #nombre de la hoja
                   index_col='ID' #nombre de la columna del indice
                  )
df['Edad_transcrito'] = pd.to_numeric(df['Edad_transcrito'], errors='coerce') #Limpieza
df

,Numero_progresivo_transcrito,Nombre_completo_transcrito,Primer_apellido,Segundo_apellido,Nombres_propios,Fecha_transcrito,Fecha_estandar,Expediente_SEMEFO_transcrito,Procedencia_transcrito,Procedencia_estandar,...,Diagnostico_estandar,Diagnostico_extendido,Sexo,Edad_transcrito,Tipo_restos,Bitacora_ingresos,Pagina_PDF,Foja_transcrito,Observaciones,Conocido_desconocido
ID,,,,,,,,,,,,,,,,,,,,,
BO_1968_00001,S-D,acosta ortega teresa,acosta,ortega,teresa,1968-01-03 00:00:00,1968-01-03,37,S-D,S-D,...,S-D,sin datos,Femenino,NaN,Cadáver,semefo_df_bo_1968,2,1,NaN,conocido
BO_1968_00002,S-D,avila de cuestas catalina,avila,de cuestas,catalina,1968-01-05 00:00:00,1968-01-05,58,S-D,S-D,...,S-D,sin datos,Femenino,NaN,Cadáver,semefo_df_bo_1968,2,1,NaN,conocido
BO_1968_00003,S-D,arzate paredes juan,arzate,paredes,juan,1968-01-07 00:00:00,1968-01-07,83,S-D,S-D,...,S-D,sin datos,Masculino,NaN,Cadáver,semefo_df_bo_1968,2,1,NaN,conocido
BO_1968_00004,S-D,alvarez martinez isaac,alvarez,martinez,isaac,1968-01-07 00:00:00,1968-01-07,86,S-D,S-D,...,S-D,sin datos,Masculino,NaN,Cadáver,semefo_df_bo_1968,2,1,NaN,conocido
BO_1968_00005,S-D,arellano viuda de campos ma.,arellano,viuda de campos,ma.,1968-01-07 00:00:00,1968-01-07,88,S-D,S-D,...,S-D,sin datos,Femenino,NaN,Cadáver,semefo_df_bo_1968,2,1,NaN,conocido
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BO_1982_07489,S-D,placenta,s-d,s-d,s-d,1982-06-05 00:00:00,1982-06-05,3079,15a,15A,...,S-D,sin datos,S-D,NaN,Miembros,semefo_df_bo_1982,251,156,NaN,desconocido
BO_1982_07490,S-D,5 dedos del pie derecho de desconocido,s-d,s-d,s-d,1982-06-05 00:00:00,1982-06-05,3060,32a,32A,...,S-D,sin datos,S-D,NaN,Miembros,semefo_df_bo_1982,251,156,NaN,desconocido
BO_1982_07491,S-D,dedo de desconocido,s-d,s-d,s-d,1982-11-19 00:00:00,1982-11-19,6389,32a,32A,...,S-D,sin datos,S-D,NaN,Miembros,semefo_df_bo_1982,251,156,NaN,desconocido


Al trabajar con conjuntos de datos ordenados es importante definir exactamente que es la observación. Por ejemplo, en el HBO se puede suponer que cada observación o fila corresponde a una persona. Sin embargo, esto no es correcto, ya que cada observación corresponde a un ingreso registrado en el SEMEFO, lo cual incluye cadáveres, miembros y fetos. Por lo tanto, en teoría puede haber el ingreso del miembro de una persona en una fecha y el posterior ingreso del cadáver de la misma persona en otra fecha, lo cual correspondería a dos observaciones, aunque sea una misma persona.

Otro caso donde es importante definir la observación es en los estudios longitudinales, donde se estudia al mismo grupo de objetos a lo largo del tiempo. Por ejemplo, si estamos estudiando el efecto de una droga en ratas a lo largo de un año y se toman mediciones todas las semanas, las observaciones no son las ratas en sí, sino las mediciones semanales, las cuales incluyen el individuo y la fecha como variables.

## 5.b Diccionario de datos

Un diccionario de datos proporciona una descripción detallada de las variables contenidas en un conjunto de datos, especificando su significado, tipo, formato y cualquier otra información relevante para su correcta interpretación y uso. El objetivo del diccionario de datos es garantizar la consistencia, claridad y estándares comunes entre todos los usuarios de los datos, facilitando su comprensión y reduciendo errores.

Un diccionario de datos debe contener.
* **Variable**: Nombre de la variable tal como aparece en el conjunto de datos.
* **Tipo**: Clasificación del tipo de dato (por ejemplo, numérico, texto, fecha, booleano, etc.).
* **Descripción**: Explicación detallada sobre el significado de la variable y cómo debe interpretarse.

Además, es útil que incluya:
* **Ejemplo**: Un valor de ejemplo representativo para ilustrar el formato y la información de la variable.
* **Formato**: Especificación del formato esperado para la variable (por ejemplo, "dd/mm/aaaa" para fechas o números con dos decimales).
* **Notas y aclaraciones**: Cualquier nota adicional o consideración especial que sea importante para la interpretación correcta de la variable.

Los estándares internacionales son normas, reglas o directrices establecidas por organismos reconocidos a nivel mundial para garantizar la interoperabilidad, calidad, seguridad y uniformidad en distintos ámbitos, como la tecnología, la industria, la salud y los datos. Estos estándares facilitan la comunicación y el intercambio de información entre sistemas, instituciones y países, asegurando que todos trabajen bajo las mismas reglas. Seguir estos estándares al estructurar los datos facilita la integración de datos provenientes de diferentes fuentes y se mejora la calidad de la información.

Algunos ejemplos de estándares internacionales:
* [ISO](www.iso.org) (International Organization for Standardization): ISO 11179 para metadatos y diccionarios de datos	
* [W3C](www.w3.org) (World Wide Web Consortium): DCAT para catalogación de datos abiertos incluyendo datos de gobierno
* [LOINC](loinc.org) y [SNOMED](www.snomed.org): SNOMED CT para terminología médica
* [DCMI](www.dublincore.org) (Dublin Core Metadata Initiative): Dublin Core para descripción de recursos digitales	

Es importante recalcar las diferencias entre diccionario de datos, metadata y diccionario de Python:
* Diccionario de Datos (de un conjunto de datos). Es un documento estructurado que describe las variables de un conjunto de datos. Incluye nombre de la variable, tipo de dato, descripción.
* Metadatos. Son datos sobre los datos, proporcionando información adicional sobre un archivo, conjunto de datos o sistema. Incluyen detalles como fuente, fecha de creación, propietario, permisos de acceso y estructura.
* Diccionario en Python (`dict`). Es una estructura de datos en Python que almacena pares clave-valor. Por ejemplo: `persona = {"nombre": "Ana", "edad": 25, "pais": "Mexico"}`


En el caso del HBO podemos consultar el diccionario de datos incluido en el archivo de Excel, el cual forma parte del conjunto de datos. El incluir el diccionario de datos permite entender la información incluida, el contexto de los datos, cómo fueron generados y sus limitaciones.

In [2]:
df_dict = pd.read_excel('data_raw/CNB_DOB_BPGS_Respuesta_Solicitud 332163723000249.xlsx', 
                        sheet_name='Diccionario de datos', header=1)
df_dict

,Variable,Tipo de variable,Descripción operativa,Aclaraciones,Formato,Ejemplo,Fundamento
0,ID,Alfanumérico,Identificador único para cada resto asignado p...,"En algunas ocasiones, se encontraron registros...",BO (corresponde a Bitácora Onomástica de Ingre...,BO_1975_00036,"Artículo 94, fracción IV de la Ley General en ..."
1,Numero_progresivo_transcrito,Numérico,Número progresivo con el cual se registraron o...,Dentro de la misma Bitácora Onomástica de Ingr...,"S-D, 1, 2, 3...",36,De acuerdo con el Manual de Procedimientos de...
2,Nombre_completo_transcrito,Texto,Nombre con el cual se registraron originalment...,"En la transcripción, se mantuvo la forma en qu...",-Primer apellido; segundo apellido; nombre (s)...,abreu tavera tomas,En la información mínima que el SEMEFO indica ...
3,Primer_apellido,Texto,Primer apellido inferido a partir del nombre.,NaN,Primer apellido,abreu,NaN
4,Segundo_apellido,Texto,Segundo apellido inferido a partir del nombre.,"En algunos casos, el dato corresponde al apell...",Segundo apellido,tavera,NaN
5,Nombres_propios,Texto,Nombres propios inferidos a partir del nombre.,"En ocasiones, en la Bitácora Onomástica de Ing...",Nombre propio,tomas,NaN
6,Fecha_transcrito,Fecha,Fecha de ingreso con la cual se registraron or...,Algunos registros solo incluyen el mes como fe...,aaaa-mm-dd,1975-02-12 00:00:00,De acuerdo con el Manual de Procedimientos del...
7,Fecha_estandar,Fecha,Fecha de ingreso con la cual se registraron or...,Cuando había información faltante o fechas ine...,aaaa-mm-dd,1975-02-12,Estandarizar la fecha transcrita en formato da...
8,Expediente_SEMEFO_transcrito,Numérico,Número de expediente interno del SEMEFO-DF con...,NaN,#,805,De acuerdo con el Manual de Procedimientos del...
9,Procedencia_transcrito,Texto,"Siglas de la procedencia (hospital, agencia de...",Las Bitácoras de 1968 a 1973 solo incluyen el ...,-#A\n-siglas del hospital o delegación\n-#a De...,CH-1,De acuerdo con el Manual de Procedimientos del...


### Ejercicio 1

Relaciona cada fila en el diccionario de datos con la variable o columna correspondiente en el conjunto de datos principal (HBO).


## 5.c Tablas pivote: relacionar variables

Las tablas pivote o tabla dinámica permiten reorganizar y resumir datos de forma que faciliten su análisis al relacionar dos o más variables. Son útiles para calcular estadísticas resumidas como sumas, medias o conteos agrupados por diferentes categorías. Esto se puede hacer en muchas hojas de cálculo, en `pandas` usaremos la función `pd.pivot_table()`.

La estructura típica de una tabla pivote se compone de las siguientes partes:

* Índice (Filas): Las filas representan las categorías principales por las que se agrupa el conjunto de datos.
* Columnas: Las columnas representan las variables o categorías adicionales por las cuales se distribuyen los datos.
* Valores: Los valores son los datos que se resumen en la intersección entre filas y columnas. Estos valores pueden ser sumas, promedios, conteos o cualquier otra función de agregación aplicada a los datos originales.
* Campos adicionales (Opcionales): Algunas tablas pivote incluyen filtros adicionales para permitir una segmentación dinámica de los datos, como seleccionar solo un rango de fechas o una categoría específica.

Por ejemplo, supongamos que se quiere ver la relación entre el tipo de restos y el sexo. Lo primero que se debe de hacer es imaginar cómo se verá la tabla, colocando las filas, columnas y valores.

|         | Mujer | Hombre | S-D |
|---------|------|------|------|
| Cadáver | ###  | ###  | ###  |
| Miembros| ###  | ###  | ###  |
| Feto    | ###  | ###  | ###  |

donde '###' es el número de ingresos de ese tipo de restos en el año

A continuación es necesario determinar donde está la información en el conjunto de datos ordenado o si no existe calcularla.

* Filas: Tipo_restos
* Columnas: Sexo
* Función: función `size()`

Basándonos en esto podemos escribir la función correspondiente.

**Nota**: La función `count()` es muy similar a `size()`, pero se aplica a cada columna y no cuenta los `nan`, por lo que los valores pueden variar.

In [3]:
# tabla pivote
pt_restos_sexo = pd.pivot_table(df, index='Tipo_restos', columns='Sexo', aggfunc='size')
pt_restos_sexo

Sexo,Femenino,Masculino,S-D
Tipo_restos,,,
Cadáver,19503,71182,569
Feto,462,662,1720
Miembros,384,1511,170
Recién nacido,251,379,3
Restos óseos,1,7,40


Es posible generar tablas pivote mucho más complicadas a partir de un conjunto de datos ordenados. Por ejemplo, supongamos que nos interesa saber si hay diferencias en la edad promedio, los de ingresos conocidos y desconocidos de cada sexo a lo largo de los años.

La tabla se vería:

|                    |  1973  |  1974  |  1975  |
|--------------------|--------|--------|--------|
| Mujer conocida     | p_edad | p_edad | p_edad |
| Mujer desconocida  | p_edad | p_edad | p_edad |
| Hombre conocido    | p_edad | p_edad | p_edad |
| Hombre desconocido | p_edad | p_edad | p_edad |

donde 'p_edad' es el promedio de edad

A continuación es necesario determinar donde está la información en el conjunto de datos ordenado. Si no existe directamente se puede calcular

* Filas: Sexo y Conocido_desconocido
* Columnas: Año obtenido de Fecha_estandar
* Values: Edad_trasncrito
* Función: `mean`

Se pueden usar varios variables o columnas para generar la tabla pivote. Esto genera un [multi-índice](https://pandas.pydata.org/docs/user_guide/advanced.html#). 

In [4]:
# Calculemos el año
df['Año'] = df['Fecha_estandar'].dt.year 
# Tabla pivote
pt_sexocon_año_edadmean = pd.pivot_table(df, 
                                index=['Sexo','Conocido_desconocido'], 
                                columns='Año', 
                                values='Edad_transcrito',
                                aggfunc='mean')
pt_sexocon_año_edadmean

Año                             1973.0     1974.0     1975.0     1976.0  \
Sexo      Conocido_desconocido                                            
Femenino  conocido                 NaN  39.101967  38.101181  39.472951   
          desconocido              NaN  43.990196  18.048649  26.474752   
Masculino conocido                53.0  36.293636  36.693664  36.977398   
          desconocido              NaN  42.531464  36.187006  36.161958   
S-D       conocido                 NaN        NaN        NaN        NaN   
          desconocido              NaN        NaN  10.400000   4.166667   

Año                                1977.0     1978.0     1979.0  1980.0  
Sexo      Conocido_desconocido                                           
Femenino  conocido              40.115475  47.376248  38.598319     NaN  
          desconocido           21.061224  19.808081  25.651934     NaN  
Masculino conocido              36.341400  41.630306  36.675401     NaN  
          desconocido           36.331971  37.511224  38.538830    35.0  
S-D       conocido               8.000000        NaN        NaN     NaN  
          desconocido           11.600000   4.500000  18.125000     NaN

Podemos volver el multi-índice en columnas con la función `.reset_index()`.

In [5]:
pt_sexocon_año_edadmean.reset_index( inplace=True )
pt_sexocon_año_edadmean

Año,Sexo,Conocido_desconocido,1973.0,1974.0,1975.0,1976.0,1977.0,1978.0,1979.0,1980.0
0,Femenino,conocido,NaN,39.101967,38.101181,39.472951,40.115475,47.376248,38.598319,NaN
1,Femenino,desconocido,NaN,43.990196,18.048649,26.474752,21.061224,19.808081,25.651934,NaN
2,Masculino,conocido,53.0,36.293636,36.693664,36.977398,36.341400,41.630306,36.675401,NaN
3,Masculino,desconocido,NaN,42.531464,36.187006,36.161958,36.331971,37.511224,38.538830,35.0
4,S-D,conocido,NaN,NaN,NaN,NaN,8.000000,NaN,NaN,NaN
5,S-D,desconocido,NaN,NaN,10.400000,4.166667,11.600000,4.500000,18.125000,NaN


Esta no es la única forma de obtener tablas derivadas, más adelante en el curso trataremos el tema de `group_by` (split-apply-combine) más a detalle.

Existen una gran cantidad de funciones que se pueden usar. Es recomendable revisar la documentación de [funciones de agrupamiento](https://pandas.pydata.org/pandas-docs/stable/reference/groupby.html#seriesgroupby-computations-descriptive-stats).
Algunas funciones útiles son: 
* mean: promedio
* median: mediana
* std: desviación estándar
* sum: suma de valores
* size: tamaño del grupo
* count: conteo del grupo
* first: primer valor
* last: último valor
* nth: n-esimo valor
* min: valor mímino
* max: valor máximo

Además, se pueden usar funciones de otras bibliotecas como `numpy` o definir funciones especiales.
Por ejemplo, la siguiente función regresa los strings más comunes de una columna.

In [6]:
def textos_mas_comunes(series, n=5, sep=', '):
    textos = series.value_counts().head(n)
    textos = textos.index
    textos = sep.join(textos)
    return textos

pt_sexocon_restos_nombrecomun = pd.pivot_table(df, 
                                   index=['Sexo','Conocido_desconocido'], 
                                   columns='Tipo_restos', 
                                   values='Nombres_propios',
                                   aggfunc=textos_mas_comunes)
pt_sexocon_restos_nombrecomun

Tipo_restos                                                               Cadáver  \
Sexo      Conocido_desconocido                                                      
Femenino  conocido                     maria, guadalupe, juana, margarita, carmen   
          desconocido                       s-d, juana, guadalupe, maria, carmela   
Masculino conocido                          jose, juan, francisco, antonio, jesus   
          desconocido                        s-d, francisco, juan, antonio, pedro   
S-D       S-D                                                                 s-d   
          conocido                          s-d, guadalupe, cruz, "n", concepcion   
          desconocido           s-d, guadalupe, loreto, feto (r.n.) 738, el al...   

Tipo_restos                    Feto       Miembros Recién nacido Restos óseos  
Sexo      Conocido_desconocido                                                 
Femenino  conocido              s-d            s-d           s-d          NaN  
          desconocido           s-d      s-d, gina           s-d          s-d  
Masculino conocido              s-d   s-d, gerardo           s-d          NaN  
          desconocido           s-d  s-d, salvador           s-d          s-d  
S-D       S-D                   NaN            NaN           NaN          NaN  
          conocido              s-d            s-d           NaN          NaN  
          desconocido           s-d            s-d           s-d          s-d

### Ejercicio 2

¿Cómo varía la edad mínima, mediana y máxima por tipo de restos y sexo?

NOTA: Para responder esta pregunta es necesario usar varias tablas pivote.

## 5.d Melt: transformar a tidy data

Una de las herramientas más usadas para el análisis de datos es Excel, ya que es muy potente y fácil de usar. Sin embargo, muchas veces al diseñar las tablas esto se hace mezclando la obtención, limpieza y análisis de datos en un solo paso, lo cual puede dificultar su posterior manejo.

Es común que las tablas de Excel no estén en formato tidy data, sino que estén en alguna variación similar a una tabla pivote. En ese caso es necesario reestructurar la tabla a formato tidy data.

Es importante destacar que existen muchos formatos de datos desordenados, por lo que puede ser necesario realizar una serie de correcciones manuales y automáticas dependiendo de la situación. El artículo original de [tidy data](http://vita.had.co.nz/papers/tidy-data.pdf) cubre varios ejemplos comunes. 
AIs generativas como ChatGPT y DeepSeek también pueden servir para obtener el código para reestructurar los datos, pero es necesario ser cuidadoso al escribir el prompt y revisar a detalle los resultados.

La función `melt` en `pandas` es útil para convertir datos en formato ancho (wide) como el de una tabla pivote a formato largo (long) cómo el de tidy data. Cuando usamos `melt`, seleccionamos una o más columnas como identificadores y convertimos las demás columnas en dos columnas nuevas: una para los nombres de las variables (`variable`) y otra para sus valores (`value`).

La función `melt` tiene los siguientes parámetros clave:
- `id_vars`: Columnas que actúan como identificadores y no deben transformarse.
- `value_vars`: Columnas que se convertirán en las nuevas columnas de `variable` y `value`. Si no se especifica, se utilizan todas las columnas que no están en `id_vars`.
- `var_name`: Nombre personalizado para la columna de variables.
- `value_name`: Nombre personalizado para la columna de valores.

Por ejemplo, supongamos que se quiere transformar la tabla pivote `pt_sexocon_año_edadmean` a una tabla en formato tidy data.

| Sexo  | Cono_descono |  1973  |  1974  |  1975  |
|-------|--------------|--------|--------|--------|
| Mujer | conocida     | p_edad | p_edad | p_edad |
| Mujer | desconocida  | p_edad | p_edad | p_edad |
| Hombre| conocido     | p_edad | p_edad | p_edad |
| Hombre| desconocido  | p_edad | p_edad | p_edad |

El primer paso es definir cuáles son las variables y cuáles son las observaciones.

* Variables: Sexo, Conocido_desconocido, Año, p_edad
* Observación: cada año se midió la edad promedio para cada grupo de personas

A continuación se determina que columnas es necesario reestructurar y cuáles no,

* No transformar (`id_vars`): Sexo, Conocido_desconocido
* Transformar (`value_vars`): años (1974, etc), es decir, las columnas restantes

También es buen momento de decidir un nombre para la columna del valor.

Basándonos en esto podemos escribir la función correspondiente. Nota cómo no definimos explicitamente `value_vars`, sino que usamos la opción por defecto, que es usar todas las columnas que NO están en `id_vars`.

In [7]:
df_edadmean = pt_sexocon_año_edadmean.melt(
                                      id_vars=['Sexo', 'Conocido_desconocido'],
                                      value_name='Edad_promedio'
                                      )
df_edadmean

,Sexo,Conocido_desconocido,Año,Edad_promedio
0,Femenino,conocido,1973.0,NaN
1,Femenino,desconocido,1973.0,NaN
2,Masculino,conocido,1973.0,53.000000
3,Masculino,desconocido,1973.0,NaN
4,S-D,conocido,1973.0,NaN
5,S-D,desconocido,1973.0,NaN
6,Femenino,conocido,1974.0,39.101967
7,Femenino,desconocido,1974.0,43.990196
8,Masculino,conocido,1974.0,36.293636
9,Masculino,desconocido,1974.0,42.531464


### Ejercicio 3

Transforma el dataframe `pt_sexocon_restos_nombrecomun` a formato tidy data.

## 5.e Concat: Unir tablas uno a uno

Es posible unir tablas de distintas maneras usando pandas, en este tutorial veremos solo las más sencillas, pero les recomendamos ver la guia de [Merge, join, concatenate and compare](https://pandas.pydata.org/docs/user_guide/merging.html).

Por ejemplo, estas series muestran la cantidad de ingresos por tipo de restos y sexo para menores y mayores de edad.

In [8]:
df_restossexo_minedad = df.loc[df['Edad_transcrito']<18, ['Tipo_restos','Sexo']] \
                          .value_counts()#.reset_index()
df_restossexo_minedad.name = 'N_menores_edad'
df_restossexo_minedad

Tipo_restos    Sexo     
Cadáver        Masculino    3109
               Femenino     1493
Feto           Masculino     409
               Femenino      303
Recién nacido  Masculino      80
               Femenino       80
Feto           S-D            44
Miembros       Masculino      29
               Femenino       17
Cadáver        S-D             1
Restos óseos   Masculino       1
Name: N_menores_edad, dtype: int64

In [9]:
df_restossexo_maxedad = df.loc[df['Edad_transcrito']>=18, ['Tipo_restos','Sexo']] \
                          .value_counts()#.reset_index()
df_restossexo_maxedad.name = 'N_mayores_edad'
df_restossexo_maxedad

Tipo_restos    Sexo     
Cadáver        Masculino    25348
               Femenino      6078
Miembros       Masculino      224
               Femenino        54
Feto           Masculino        6
Cadáver        S-D              5
Feto           Femenino         4
               S-D              1
Recién nacido  Femenino         1
Restos óseos   Masculino        1
Name: N_mayores_edad, dtype: int64

Una forma de unir dos tablas es con la función `concat()`. Esta función toma una lista de dataframes y las une por el índice o las columnas. En este caso es una unión uno a uno, es decir, cada valor del índice o columna es único y no se repiten.
Es importante notar que, aunque los valores no se repiten, los índices sí se repiten, por ejemplo ('Feto', 'Masculino') aparece dos veces.
Esta operación puede ser vista como apilar las tablas a lo alto (filas), uniendo a lo ancho (columnas).

In [10]:
pd.concat([df_restossexo_minedad,df_restossexo_maxedad])

Tipo_restos    Sexo     
Cadáver        Masculino     3109
               Femenino      1493
Feto           Masculino      409
               Femenino       303
Recién nacido  Masculino       80
               Femenino        80
Feto           S-D             44
Miembros       Masculino       29
               Femenino        17
Cadáver        S-D              1
Restos óseos   Masculino        1
Cadáver        Masculino    25348
               Femenino      6078
Miembros       Masculino      224
               Femenino        54
Feto           Masculino        6
Cadáver        S-D              5
Feto           Femenino         4
               S-D              1
Recién nacido  Femenino         1
Restos óseos   Masculino        1
dtype: int64

En segundo lugar, uniremos usando `pd.concat(axis=1)`, es decir, los valores se unirán en las filas.
Nota como los nombres de las columnas se mantuvieron como estaban en las tablas originales.
Esta operación puede ser vista como pegar las tablas a lo ancho (columnas), uniendo a lo alto (filas).

In [11]:
pd.concat([df_restossexo_minedad,df_restossexo_maxedad], axis=1)

N_menores_edad  N_mayores_edad
Tipo_restos   Sexo                                     
Cadáver       Masculino            3109         25348.0
              Femenino             1493          6078.0
Feto          Masculino             409             6.0
              Femenino              303             4.0
Recién nacido Masculino              80             NaN
              Femenino               80             1.0
Feto          S-D                    44             1.0
Miembros      Masculino              29           224.0
              Femenino               17            54.0
Cadáver       S-D                     1             5.0
Restos óseos  Masculino               1             1.0

## 5.f Merge: Unir tablas muchos a uno

Otro caso de unir tablas es cuando tenemos muchos valores que se unirán a uno solo.
Por ejemplo, supongamos que tenemos la siguiente tabla con información de los diagnósticos más comunes. Esta tabla nos da información adicional que nos gustaría unir a la información contenida en el HBO. 

In [12]:
data_diag = [['S-D','sin datos','SD'],
             ['TM','traumatismo multiple','Trauma'],
             ['TCE','traumatismo craneo encefalico','Trauma'],
             ['BN','bronconeumonia','Neumonías'],
             ['CVG','congestion visceral generalizada','Falla de órganos'],
             ['HPAFC','herida arma fuego craneo','Herida por arma de fuego'],
             ['Dispensa','dispensa','SD'],
             ['AOVA','asfixia por obstruccion de vias aereas','Obstrucción de vías aéreas'],
             ['CH','cirrosis hepatica','Cirrosis'], ]
data_diag = pd.DataFrame(data_diag, columns=['Diagnostico','Extendido','Clasificación'])
data_diag

,Diagnostico,Extendido,Clasificación
0,S-D,sin datos,SD
1,TM,traumatismo multiple,Trauma
2,TCE,traumatismo craneo encefalico,Trauma
3,BN,bronconeumonia,Neumonías
4,CVG,congestion visceral generalizada,Falla de órganos
5,HPAFC,herida arma fuego craneo,Herida por arma de fuego
6,Dispensa,dispensa,SD
7,AOVA,asfixia por obstruccion de vias aereas,Obstrucción de vías aéreas
8,CH,cirrosis hepatica,Cirrosis


Por otro lado, podemos calcular los diagnósticos más comunes:

In [13]:
df_diag = df['Diagnostico_transcrito'].value_counts().head(10).reset_index()
df_diag.columns = ['Diagnostico','Freq']
df_diag

,Diagnostico,Freq
0,S-D,55878
1,TM,7505
2,TCE,5939
3,BN,1826
4,CVG,1287
5,TCT,1176
6,HPAFPC,833
7,Dispensa,827
8,AOVA,791
9,BN+CH,715


Ahora, ambas tablas comparten la columna 'Diagnostico', por lo que es posible unirlas en una sola.

Existen varias formas de unir tablas:
* **`inner` (por defecto)**: Devuelve las filas que coinciden en ambas tablas.
* **`outer`**: Devuelve todas las filas, rellenando con valores `NaN` donde no hay coincidencias.
* **`left`**: Devuelve todas las filas de la tabla izquierda y las coincidencias de la tabla derecha.
* **`right`**: Devuelve todas las filas de la tabla derecha y las coincidencias de la tabla izquierda.

En este caso usaremos la opción `how='outer'`. Nota como la tabla resultante mide 12 x 4. Las cuatro columnas son la unión de la tabla de frecuencia de diagnósticos más la tabla con los diagnósticos extendidos. Por otro lado, las 12 columnas son el resultado de que algunos diagnósticos aparecen solo en la tabla de frecuencias (TCT, HPAFPC, BN+CH) y otros solo aparecen en la tabla extendida (HPAFC, CH). En los casos donde no hay 

In [14]:
df_diag.merge(data_diag, how='outer', on='Diagnostico')

,Diagnostico,Freq,Extendido,Clasificación
0,AOVA,791.0,asfixia por obstruccion de vias aereas,Obstrucción de vías aéreas
1,BN,1826.0,bronconeumonia,Neumonías
2,BN+CH,715.0,NaN,NaN
3,CH,NaN,cirrosis hepatica,Cirrosis
4,CVG,1287.0,congestion visceral generalizada,Falla de órganos
5,Dispensa,827.0,dispensa,SD
6,HPAFC,NaN,herida arma fuego craneo,Herida por arma de fuego
7,HPAFPC,833.0,NaN,NaN
8,S-D,55878.0,sin datos,SD
9,TCE,5939.0,traumatismo craneo encefalico,Trauma


Existen muchas formas de unir tablas usando llaves, una vez más recomendamos ver el tutorial de [merge, join, concatenate and compare](https://pandas.pydata.org/docs/user_guide/merging.html).

### Ejercicio 4:
Une `df_diag` y `data_diag` con la función `merge()` usando los parámetros: `inner`, `left` y `right`. Para cada caso detalla el tamaño de la tabla resultante y qué diagnósticos de qué tabla faltan.

## 5.g Ejemplo: Reportes de adopción DIF

Realizaremos un pequeño ejemplo de cómo reestructurar un conjunto de datos para que quede en formato tidy data.

En este caso usaremos las [estadísticas de adopción](https://datos.gob.mx/busca/dataset/estadistica-de-adopcion) de 2024 y 2024, las cuales son publicadas por el gobierno de México.
Cómo su nombre lo indica, estos archivos no son en sí un conjunto de datos ordenados, sino estadísticas resumidas.

Revisemos uno de los archivos:

In [15]:
df_adop_24_2 = pd.read_excel('data_raw/Adopciones_JULIO-DICIEMBRE_2024.xlsx')
df_adop_24_2

,Reporte Semestral de Adopción,Periodo,Institución Encargada del Trámite,Solicitudes Recibidas,Solicitudes en Trámite,Solicitudes de adopción con resolución sobre la no emisión del Certificado de Idoneidad,Solicitudes de adopción con resolución sobre la improcedencia,Adopciones concluidas,Femenino,Masculino,...,Niñas de \n5-8 años 11 meses,Niñas y adolescentes mujeres de \n9 -18 años,Total de Niñas,Niños de 0 - 4 años 11 meses,Niños de 5-8 años 11 meses,Niños y adolescentes hombre de 9 - 13 años,Niños y adolescentes hombres de 14 - 18 años,Total de Niños,Estado de Origen,Estado de Receptor
0,Nacional,Julio-Diciembre 2024,DIF Nacional,41,38,0,0,2,1,1,...,0,1,1,1,0,0,0,1,Ciudad de México,Ciudad de México
1,Internacional,Julio-Diciembre 2024,DIF Nacional,3,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN


Este archivo es un resumen estadístico, las filas son si la adopción es Nacional o Internacional, mientras que las columnas incluyen información sobre el semestre, la institución, las solicitudes, y los grupos agregados a los que pertenecen los infantes adoptados.

El primer paso es definir la pregunta de investigación:
¿Cuáles son las tendencias en la adopción con respecto a las características de los menores de edad adoptados?

El segundo paso es definir las variables que nos interesan 
* Sexo
* Edad (grupo de edad)
* Periodo
* Nacional o internacional
* Institución

El tercer paso es definir las observaciones. En este caso no estamos observando a individuos separados, sino a grupos. Esto es el resultado de los datos de origen, pero tiene la ventaja de que nos permite proteger los datos personales de los menores (aunque estrictamente hablando no garantiza anonimidad estadística). Entonces, cada observación son el conjunto de menores de edad adoptados por semestre y agrupados por sexo y grupo de edad.

A partir de esto tenemos una idea más clara de cómo nos gustaría que fuera la tabla final:

. | Sexo | Edad | Periodo | Nivel | Institución | Adopciones
--|------|------|---------|-------|-------------|-----------
0 | Masculino | 0-4 | Enero-Junio 2024 | Nacional | DIF Nacional | 1
1 | Femenino | 0-4 | Enero-Junio 2024 | Nacional | DIF Nacional | 0

Sin embargo, hay dos problemas adicionales:
* Las variables de Sexo y Edad se encuentran unificadas en el nombre de columna
* Los datos se encuentran en varios archivos separados

Trataremos de reestructurar la tabla para que sea lo más parecido a la tabla final que queremos.

In [16]:
df_adop_24_2.columns

Index(['Reporte Semestral de Adopción ', 'Periodo',
       'Institución Encargada del Trámite', 'Solicitudes Recibidas',
       'Solicitudes  en Trámite',
       'Solicitudes de adopción con resolución sobre la no emisión del Certificado de Idoneidad',
       'Solicitudes de adopción con resolución sobre la improcedencia',
       'Adopciones concluidas', 'Femenino', 'Masculino ',
       'Niñas de 0 - 4 años 11 meses', 'Niñas de \n5-8 años 11 meses',
       'Niñas y adolescentes mujeres de  \n9 -18 años', 'Total de Niñas',
       'Niños de 0 - 4 años 11 meses', 'Niños de  5-8 años 11 meses',
       'Niños y adolescentes hombre de 9 - 13 años ',
       'Niños y adolescentes hombres de 14 - 18 años ', 'Total de Niños',
       'Estado de Origen ', 'Estado de Receptor'],
      dtype='object')

In [17]:
col_sexoedad = ['Niñas de 0 - 4 años 11 meses', 'Niñas de \n5-8 años 11 meses',
                'Niñas y adolescentes mujeres de  \n9 -18 años',
                'Niños de 0 - 4 años 11 meses', 'Niños de  5-8 años 11 meses',
                'Niños y adolescentes hombre de 9 - 13 años ',
                'Niños y adolescentes hombres de 14 - 18 años ']
data_24_2 = df_adop_24_2.melt(
                              id_vars=['Reporte Semestral de Adopción ', 'Periodo', 'Institución Encargada del Trámite'],
                              value_vars=col_sexoedad
                             )
data_24_2

,Reporte Semestral de Adopción,Periodo,Institución Encargada del Trámite,variable,value
0,Nacional,Julio-Diciembre 2024,DIF Nacional,Niñas de 0 - 4 años 11 meses,0
1,Internacional,Julio-Diciembre 2024,DIF Nacional,Niñas de 0 - 4 años 11 meses,0
2,Nacional,Julio-Diciembre 2024,DIF Nacional,Niñas de \n5-8 años 11 meses,0
3,Internacional,Julio-Diciembre 2024,DIF Nacional,Niñas de \n5-8 años 11 meses,0
4,Nacional,Julio-Diciembre 2024,DIF Nacional,Niñas y adolescentes mujeres de \n9 -18 años,1
5,Internacional,Julio-Diciembre 2024,DIF Nacional,Niñas y adolescentes mujeres de \n9 -18 años,0
6,Nacional,Julio-Diciembre 2024,DIF Nacional,Niños de 0 - 4 años 11 meses,1
7,Internacional,Julio-Diciembre 2024,DIF Nacional,Niños de 0 - 4 años 11 meses,0
8,Nacional,Julio-Diciembre 2024,DIF Nacional,Niños de 5-8 años 11 meses,0
9,Internacional,Julio-Diciembre 2024,DIF Nacional,Niños de 5-8 años 11 meses,0


Cambiar los nombres de columnas

In [18]:
data_24_2.columns = ['Nivel','Periodo','Institución','SexoEdad','Cantidad']
data_24_2

,Nivel,Periodo,Institución,SexoEdad,Cantidad
0,Nacional,Julio-Diciembre 2024,DIF Nacional,Niñas de 0 - 4 años 11 meses,0
1,Internacional,Julio-Diciembre 2024,DIF Nacional,Niñas de 0 - 4 años 11 meses,0
2,Nacional,Julio-Diciembre 2024,DIF Nacional,Niñas de \n5-8 años 11 meses,0
3,Internacional,Julio-Diciembre 2024,DIF Nacional,Niñas de \n5-8 años 11 meses,0
4,Nacional,Julio-Diciembre 2024,DIF Nacional,Niñas y adolescentes mujeres de \n9 -18 años,1
5,Internacional,Julio-Diciembre 2024,DIF Nacional,Niñas y adolescentes mujeres de \n9 -18 años,0
6,Nacional,Julio-Diciembre 2024,DIF Nacional,Niños de 0 - 4 años 11 meses,1
7,Internacional,Julio-Diciembre 2024,DIF Nacional,Niños de 0 - 4 años 11 meses,0
8,Nacional,Julio-Diciembre 2024,DIF Nacional,Niños de 5-8 años 11 meses,0
9,Internacional,Julio-Diciembre 2024,DIF Nacional,Niños de 5-8 años 11 meses,0


A continuación convertiremos la columna de `SexoEdad` en dos columnas separadas de `Sexo` y `Edad`.
Este proceso puede ser complicado, ya que hay varias situaciones tipográficas (espacios extra, saltos de línea, grupos no iguales). Hay varias formas de hacer esto.

Se puede usar un diccionario de sustitución:

In [19]:
replace_edad = {'Niñas de 0 - 4 años 11 meses':'0-4', 
                'Niñas de \n5-8 años 11 meses':'5-8',
                'Niñas y adolescentes mujeres de  \n9 -18 años':'9-18',
                'Niños de 0 - 4 años 11 meses':'0-4', 
                'Niños de  5-8 años 11 meses':'5-8',
                'Niños y adolescentes hombre de 9 - 13 años ':'9-13',
                'Niños y adolescentes hombres de 14 - 18 años ':'14-18',}
data_24_2['Edad'] = data_24_2['SexoEdad'].replace(replace_edad)
data_24_2

,Nivel,Periodo,Institución,SexoEdad,Cantidad,Edad
0,Nacional,Julio-Diciembre 2024,DIF Nacional,Niñas de 0 - 4 años 11 meses,0,0-4
1,Internacional,Julio-Diciembre 2024,DIF Nacional,Niñas de 0 - 4 años 11 meses,0,0-4
2,Nacional,Julio-Diciembre 2024,DIF Nacional,Niñas de \n5-8 años 11 meses,0,5-8
3,Internacional,Julio-Diciembre 2024,DIF Nacional,Niñas de \n5-8 años 11 meses,0,5-8
4,Nacional,Julio-Diciembre 2024,DIF Nacional,Niñas y adolescentes mujeres de \n9 -18 años,1,9-18
5,Internacional,Julio-Diciembre 2024,DIF Nacional,Niñas y adolescentes mujeres de \n9 -18 años,0,9-18
6,Nacional,Julio-Diciembre 2024,DIF Nacional,Niños de 0 - 4 años 11 meses,1,0-4
7,Internacional,Julio-Diciembre 2024,DIF Nacional,Niños de 0 - 4 años 11 meses,0,0-4
8,Nacional,Julio-Diciembre 2024,DIF Nacional,Niños de 5-8 años 11 meses,0,5-8
9,Internacional,Julio-Diciembre 2024,DIF Nacional,Niños de 5-8 años 11 meses,0,5-8


O usar condiciones para asignar valores

In [20]:
data_24_2['Sexo'] = None
data_24_2.loc[ data_24_2['SexoEdad'].str.contains('Niñas'), 'Sexo' ] = 'Femenino'
data_24_2.loc[ data_24_2['SexoEdad'].str.contains('Niños'), 'Sexo' ] = 'Masculino'
data_24_2

,Nivel,Periodo,Institución,SexoEdad,Cantidad,Edad,Sexo
0,Nacional,Julio-Diciembre 2024,DIF Nacional,Niñas de 0 - 4 años 11 meses,0,0-4,Femenino
1,Internacional,Julio-Diciembre 2024,DIF Nacional,Niñas de 0 - 4 años 11 meses,0,0-4,Femenino
2,Nacional,Julio-Diciembre 2024,DIF Nacional,Niñas de \n5-8 años 11 meses,0,5-8,Femenino
3,Internacional,Julio-Diciembre 2024,DIF Nacional,Niñas de \n5-8 años 11 meses,0,5-8,Femenino
4,Nacional,Julio-Diciembre 2024,DIF Nacional,Niñas y adolescentes mujeres de \n9 -18 años,1,9-18,Femenino
5,Internacional,Julio-Diciembre 2024,DIF Nacional,Niñas y adolescentes mujeres de \n9 -18 años,0,9-18,Femenino
6,Nacional,Julio-Diciembre 2024,DIF Nacional,Niños de 0 - 4 años 11 meses,1,0-4,Masculino
7,Internacional,Julio-Diciembre 2024,DIF Nacional,Niños de 0 - 4 años 11 meses,0,0-4,Masculino
8,Nacional,Julio-Diciembre 2024,DIF Nacional,Niños de 5-8 años 11 meses,0,5-8,Masculino
9,Internacional,Julio-Diciembre 2024,DIF Nacional,Niños de 5-8 años 11 meses,0,5-8,Masculino


Quitemos la columna sobrante y reordenemos

In [21]:
data_24_2 = data_24_2[['Sexo', 'Edad', 'Periodo', 'Nivel', 'Institución', 'Cantidad']] 
data_24_2

,Sexo,Edad,Periodo,Nivel,Institución,Cantidad
0,Femenino,0-4,Julio-Diciembre 2024,Nacional,DIF Nacional,0
1,Femenino,0-4,Julio-Diciembre 2024,Internacional,DIF Nacional,0
2,Femenino,5-8,Julio-Diciembre 2024,Nacional,DIF Nacional,0
3,Femenino,5-8,Julio-Diciembre 2024,Internacional,DIF Nacional,0
4,Femenino,9-18,Julio-Diciembre 2024,Nacional,DIF Nacional,1
5,Femenino,9-18,Julio-Diciembre 2024,Internacional,DIF Nacional,0
6,Masculino,0-4,Julio-Diciembre 2024,Nacional,DIF Nacional,1
7,Masculino,0-4,Julio-Diciembre 2024,Internacional,DIF Nacional,0
8,Masculino,5-8,Julio-Diciembre 2024,Nacional,DIF Nacional,0
9,Masculino,5-8,Julio-Diciembre 2024,Internacional,DIF Nacional,0


Ahora, es necesario hacer lo mismo para todos los dataframes. En este caso todos tienen la misma estructura, lo cual facilita el procesamiento.

In [22]:

# En este diccionario iremos guardando cada una de las tablas 
data_adop = dict()

filenames = ['Adopciones_ENERO-JUNIO_2023.xlsx',  'Adopciones_JUL-DIC_2023.xlsx',  
             'Adopciones_JULIO-DICIEMBRE_2024.xlsx',  'Datos_Abiertos_Adopciones_ENERO_JUNIO_2024.xlsx']
# iterar sobre los archivos
for file in filenames:
    # que archivo estamos procesando
    print(file)
    # abrir el archivo
    data = pd.read_excel('data_raw/'+file)
    # transformar con melt
    data = data.melt(
                     id_vars=['Reporte Semestral de Adopción ', 'Periodo', 'Institución Encargada del Trámite'],
                     value_vars=col_sexoedad
                     )
    # cambiar nombres de columnas
    data.columns = ['Nivel','Periodo','Institución','SexoEdad','Cantidad']
    # obtener edad con el diccionario de remplazo que definimos arriba
    data['Edad'] = data['SexoEdad'].replace(replace_edad)
    # Obtener sexo usando el texto de la columna SexoEdad
    data['Sexo'] = None
    data.loc[ data['SexoEdad'].str.contains('Niñas'), 'Sexo' ] = 'Femenino'
    data.loc[ data['SexoEdad'].str.contains('Niños'), 'Sexo' ] = 'Masculino'
    # Quitar columnas extra y reordenar
    data = data[['Sexo', 'Edad', 'Periodo', 'Nivel', 'Institución', 'Cantidad', ]]
    # Agregar al diccionario
    data_adop[file] = data

data_adop

Adopciones_ENERO-JUNIO_2023.xlsx
Adopciones_JUL-DIC_2023.xlsx
Adopciones_JULIO-DICIEMBRE_2024.xlsx
Datos_Abiertos_Adopciones_ENERO_JUNIO_2024.xlsx


{'Adopciones_ENERO-JUNIO_2023.xlsx':          Sexo   Edad             Periodo          Nivel   Institución  \
 0    Femenino    0-4  Enero - Junio 2023       Nacional  DIF Nacional   
 1    Femenino    0-4  Enero - Junio 2023  Internacional  DIF Nacional   
 2    Femenino    5-8  Enero - Junio 2023       Nacional  DIF Nacional   
 3    Femenino    5-8  Enero - Junio 2023  Internacional  DIF Nacional   
 4    Femenino   9-18  Enero - Junio 2023       Nacional  DIF Nacional   
 5    Femenino   9-18  Enero - Junio 2023  Internacional  DIF Nacional   
 6   Masculino    0-4  Enero - Junio 2023       Nacional  DIF Nacional   
 7   Masculino    0-4  Enero - Junio 2023  Internacional  DIF Nacional   
 8   Masculino    5-8  Enero - Junio 2023       Nacional  DIF Nacional   
 9   Masculino    5-8  Enero - Junio 2023  Internacional  DIF Nacional   
 10  Masculino   9-13  Enero - Junio 2023       Nacional  DIF Nacional   
 11  Masculino   9-13  Enero - Junio 2023  Internacional  DIF Nacional   
 1

Ahora uniremos todas las tablas en una sola con `concat`

In [23]:
data_adop = pd.concat(data_adop)
# regenerar el indice
data_adop.index = range(data_adop.shape[0])
data_adop

,Sexo,Edad,Periodo,Nivel,Institución,Cantidad
0,Femenino,0-4,Enero - Junio 2023,Nacional,DIF Nacional,1
1,Femenino,0-4,Enero - Junio 2023,Internacional,DIF Nacional,0
2,Femenino,5-8,Enero - Junio 2023,Nacional,DIF Nacional,2
3,Femenino,5-8,Enero - Junio 2023,Internacional,DIF Nacional,0
4,Femenino,9-18,Enero - Junio 2023,Nacional,DIF Nacional,1
5,Femenino,9-18,Enero - Junio 2023,Internacional,DIF Nacional,0
6,Masculino,0-4,Enero - Junio 2023,Nacional,DIF Nacional,0
7,Masculino,0-4,Enero - Junio 2023,Internacional,DIF Nacional,0
8,Masculino,5-8,Enero - Junio 2023,Nacional,DIF Nacional,2
9,Masculino,5-8,Enero - Junio 2023,Internacional,DIF Nacional,0


Esta tabla nos permite obtener datos que antes no era posible, por ejemplo, las adopciones por grupo de edad a lo largo del tiempo.

In [24]:
pd.pivot_table(data_adop, index='Edad', columns='Periodo', values='Cantidad', aggfunc='sum')


Periodo,Enero - Junio 2023,Enero-Junio 2024,Julio - Diciembre 2023,Julio-Diciembre 2024
Edad,,,,
0-4,1,1,7,1
14-18,0,0,1,0
5-8,4,0,8,0
9-13,0,0,1,0
9-18,1,1,1,1


## 5.e Resumen

En esta lección hemos aprendido varios conceptos:
* Repaso de conjuntos de datos ordenados o tidy data
    * Cada observación forma una fila.
    * Cada variable forma una columna.
* Diccionario de datos: descripción detallada de las variables del conjunto de datos. Para cada variable incluir:
    * Nombre de variable
    * Tipo
    * Descripción
    * Nota: es diferente a metadata o a diccionario de Python
* Tablas pivote: resumen y reorganizan datos
    * Índice: categoría principal
    * Columnas: categoría adicional
    * Valores: datos a resumir
    * Función: función para resumir los datos
    * Nota: se puede usar un multi-índice para incorporar variables
* Funciones útiles: `.mean()`, `.median()`, `.std()`, `.sum()`, `.size()`, `.count()`, `.first()`, `.last()`, `.nth()`, `.min()`, `.max()`
* Melt: tablas pivote a tidy data
    * `id_vars`: Columnas que actúan como identificadores y no deben transformarse.
    * `value_vars`: Columnas que se convertirán en las nuevas columnas de variable y value. Si no se especifica, se utilizan todas las columnas que no están en id_vars.
* Concat: unir tablas uno a uno dependiendo del eje
* Merge: unir tablas muchos a uno
    * Determinar que columna unir en cada tabla
    * En la tabla de catálogo la columna a unir debe de tener valores únicos
    * Existen varias formas de unir tablas: `inner`, `outer`, `left`, `right`

**¡Gracias!**